In [23]:
import json

import tweepy
from tweepy import OAuthHandler
from tweepy import Stream,StreamListener
from kafka import KafkaProducer, KafkaConsumer
import time
from datetime import datetime, timedelta

In [24]:
def normalize_timestamp(time):
    mytime = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    mytime += timedelta(hours=1)
    return (mytime.strftime('%Y-%m-%d %H:%M:%S'))

def on_send_success(record_metadata):
    # print(record_metadata.topic)
    print(record_metadata)
    # print(record_metadata.offset)

def on_send_error(excp):
    print("error", excp)
    log.error('I am an errback', exc_info=excp)

In [25]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],api_version=(3, 0, 0), value_serializer=lambda x: x.encode('utf-8'))
topic_name = 'tugas-11-2'

In [26]:
print(producer.bootstrap_connected())

True


In [27]:
class Listener(StreamListener):
  def __init__(self, time_limit=60):
    self.start_time = time.time()
    self.limit = time_limit
    self.saveFile = open('tweet.json', 'a')
    super(Listener, self).__init__()

  def on_data(self, data):
    if (time.time() - self.start_time) < self.limit:
      self.saveFile.write(data)
      self.saveFile.write('\n')
      producer.send(topic_name, data).add_callback(on_send_success).add_errback(on_send_error)
      return True
    else:
      self.saveFile.close()
      return False

In [28]:
consumer_key = "TCHVbt5fbcttUMjNoNYPLsyzZ"
consumer_secret = "wih7b5RUNRkezZouANXqm1aXfTq6A9SOcIcQxJ4WY5tW1b5WT0"
access_token = "1000362437558743041-VnnlqzolYZfdp843j3ZaLVZ1oAnbiK"
access_token_secret = "lmqiLv45HT3ckrnLAz0XaYdsBamkxzpszDRpbS9NeLAT5"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [29]:
stream = Stream(auth, Listener(time_limit=30))
stream.filter(track=['Shopee'])

RecordMetadata(topic='tugas-11-2', partition=0, topic_partition=TopicPartition(topic='tugas-11-2', partition=0), offset=72, timestamp=1638093806841, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=3064, serialized_header_size=-1)
RecordMetadata(topic='tugas-11-2', partition=1, topic_partition=TopicPartition(topic='tugas-11-2', partition=1), offset=56, timestamp=1638093807386, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=4904, serialized_header_size=-1)
RecordMetadata(topic='tugas-11-2', partition=0, topic_partition=TopicPartition(topic='tugas-11-2', partition=0), offset=73, timestamp=1638093808215, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=5196, serialized_header_size=-1)
RecordMetadata(topic='tugas-11-2', partition=0, topic_partition=TopicPartition(topic='tugas-11-2', partition=0), offset=74, timestamp=1638093808395, log_start_offset=0, checksum=None, serialized_key_size=-1, ser